In [5]:
import random
import csv


In [2]:
# List of supermarket items with their prices
items = [
    ("Diapers", 10.50),
    ("T-shirts", 7.00),
    ("Milk (1L)", 1.25),
    ("Bread (Whole Wheat)", 2.50),
    ("Shampoo", 4.75),
    ("Toothpaste", 3.00),
    ("Eggs (Dozen)", 2.00),
    ("Bananas (1kg)", 1.10),
    ("Rice (5kg)", 15.00),
    ("Canned Beans (400g)", 1.20)
]

In [4]:
# Function to generate a random transaction
def generate_transaction(transaction_id):
    num_items = random.randint(1, 5)  # Each transaction will have 1 to 5 items
    selected_items = random.sample(items, num_items)
    item_names = [item[0] for item in selected_items]
    total_amount = sum(item[1] for item in selected_items)

    return {
        "Transaction ID": transaction_id,
        "Items": ", ".join(item_names),
        "Total Amount": round(total_amount, 2)
    }

In [6]:
# Generate 20 transactions
transactions = [generate_transaction(i + 1) for i in range(20)]

In [7]:
# Save transactions to CSV
csv_file = "supermarket_transactions.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["Transaction ID", "Items", "Total Amount"])
    writer.writeheader()
    writer.writerows(transactions)

print(f"Transactions saved to {csv_file}")

Transactions saved to supermarket_transactions.csv


In [8]:
# Function to create a database and save to CSV
def create_database(file_name, transaction_count=20):
    transactions = [generate_transaction(i + 1) for i in range(transaction_count)]

    # Save to CSV
    with open(file_name, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=["Transaction ID", "Items", "Total Amount"])
        writer.writeheader()
        writer.writerows(transactions)
    print(f"Database saved to {file_name}")

# Create 5 different databases
for i in range(5):
    file_name = f"supermarket_transactions_{i+1}.csv"
    create_database(file_name)


Database saved to supermarket_transactions_1.csv
Database saved to supermarket_transactions_2.csv
Database saved to supermarket_transactions_3.csv
Database saved to supermarket_transactions_4.csv
Database saved to supermarket_transactions_5.csv


Each of these files contains unique sets of transactions with random items selected from the list, and the total amounts are calculated based on the prices of those items.

To implement the brute force method for generating frequent itemsets and association rules, we follow these steps:

1) Input Transaction Data: Collect or simulate transaction data where each transaction is a list of items purchased.

2) Generate All Possible Itemsets: For each transaction, generate all possible 1-itemsets, 2-itemsets, and so on, using combinations.

3) Check Frequency of Each Itemset: For each generated itemset, check how often it appears in the transaction database. Itemsets that meet or exceed a minimum frequency (support) are considered frequent.

4) Stop When No Frequent k-itemsets are Found: Continue until no frequent k-itemsets can be found, at which point the process stops.

5) Generate Association Rules: For each frequent itemset, generate association rules, which help identify how items are related.

Steps in the Brute Force Method:

Simulate or Load Transaction Data Here, we simulate transaction data similar to what would be collected in a supermarket.

Generate Itemsets For n items in the supermarket, the number of combinations increases exponentially with the number of items (k) in the itemset.

Count Support (Frequency) For each candidate itemset, count the number of transactions in which it appears and compare it to a minimum support threshold.

Generate Association Rules Once the frequent itemsets are identified, generate rules like "if A, then B" and evaluate them based on metrics like confidence.

In [9]:
import itertools
from collections import defaultdict

In [10]:
# Sample transactions (20 transactions as an example)
transactions = [
    ['Milk', 'Diapers', 'Bread'],
    ['Milk', 'Shampoo', 'Eggs'],
    ['Bread', 'Rice', 'Bananas'],
    ['Milk', 'Bananas', 'Eggs'],
    ['Toothpaste', 'Canned Beans', 'Diapers'],
    ['Milk', 'Bananas', 'Bread'],
    ['Diapers', 'T-shirts'],
    ['Rice', 'Bananas', 'Milk'],
    ['Eggs', 'Bread', 'Shampoo'],
    ['Rice', 'Bananas', 'Canned Beans'],
    ['Toothpaste', 'Milk', 'Eggs'],
    ['Bread', 'Canned Beans', 'Shampoo'],
    ['Diapers', 'Bananas', 'Milk'],
    ['Eggs', 'Milk', 'Toothpaste'],
    ['Rice', 'T-shirts', 'Bread'],
    ['Shampoo', 'Canned Beans'],
    ['Rice', 'Eggs', 'Bananas'],
    ['Milk', 'Toothpaste'],
    ['Shampoo', 'Rice', 'Diapers'],
    ['Canned Beans', 'Eggs', 'Bananas']
]

In [11]:
# Minimum support threshold
min_support = 3

# Function to generate itemsets and calculate frequency
def brute_force_itemsets(transactions, min_support):
    # Get unique items
    all_items = sorted(set(item for transaction in transactions for item in transaction))

    # Function to count the frequency of a given itemset
    def count_frequency(itemset, transactions):
        count = sum(1 for transaction in transactions if set(itemset).issubset(set(transaction)))
        return count

    # Initialize variables
    frequent_itemsets = []
    k = 1

    while True:
        # Generate k-itemsets
        itemsets = list(itertools.combinations(all_items, k))
        print(f"\nGenerating {k}-itemsets: {len(itemsets)} possible combinations")

        # Calculate support for each itemset
        current_frequent_itemsets = []
        for itemset in itemsets:
            frequency = count_frequency(itemset, transactions)
            if frequency >= min_support:
                current_frequent_itemsets.append((itemset, frequency))

        # If no frequent itemsets are found, terminate
        if not current_frequent_itemsets:
            break

        # Add frequent itemsets to the final list
        frequent_itemsets.extend(current_frequent_itemsets)
        print(f"Frequent {k}-itemsets: {current_frequent_itemsets}")

        # Increment k
        k += 1

    return frequent_itemsets


In [12]:
# Generate frequent itemsets using brute force
frequent_itemsets = brute_force_itemsets(transactions, min_support)

# Function to generate association rules
def generate_association_rules(frequent_itemsets, transactions, min_confidence=0.7):
    rules = []
    transaction_count = len(transactions)

    for itemset, support in frequent_itemsets:
        if len(itemset) < 2:
            continue

        # Generate rules of form A -> B
        for i in range(1, len(itemset)):
            for lhs in itertools.combinations(itemset, i):
                rhs = set(itemset) - set(lhs)
                lhs_count = sum(1 for transaction in transactions if set(lhs).issubset(set(transaction)))
                confidence = support / lhs_count

                if confidence >= min_confidence:
                    rules.append({
                        "rule": f"{lhs} -> {rhs}",
                        "support": support / transaction_count,
                        "confidence": confidence
                    })

    return rules

# Generate association rules
association_rules = generate_association_rules(frequent_itemsets, transactions)

# Display results
print("\nFrequent Itemsets:")
for itemset, support in frequent_itemsets:
    print(f"Itemset: {itemset}, Support: {support}")

print("\nAssociation Rules:")
for rule in association_rules:
    print(f"Rule: {rule['rule']}, Support: {rule['support']:.2f}, Confidence: {rule['confidence']:.2f}")


Generating 1-itemsets: 10 possible combinations
Frequent 1-itemsets: [(('Bananas',), 8), (('Bread',), 6), (('Canned Beans',), 5), (('Diapers',), 5), (('Eggs',), 7), (('Milk',), 9), (('Rice',), 6), (('Shampoo',), 5), (('Toothpaste',), 4)]

Generating 2-itemsets: 45 possible combinations
Frequent 2-itemsets: [(('Bananas', 'Eggs'), 3), (('Bananas', 'Milk'), 4), (('Bananas', 'Rice'), 4), (('Eggs', 'Milk'), 4), (('Milk', 'Toothpaste'), 3)]

Generating 3-itemsets: 120 possible combinations

Frequent Itemsets:
Itemset: ('Bananas',), Support: 8
Itemset: ('Bread',), Support: 6
Itemset: ('Canned Beans',), Support: 5
Itemset: ('Diapers',), Support: 5
Itemset: ('Eggs',), Support: 7
Itemset: ('Milk',), Support: 9
Itemset: ('Rice',), Support: 6
Itemset: ('Shampoo',), Support: 5
Itemset: ('Toothpaste',), Support: 4
Itemset: ('Bananas', 'Eggs'), Support: 3
Itemset: ('Bananas', 'Milk'), Support: 4
Itemset: ('Bananas', 'Rice'), Support: 4
Itemset: ('Eggs', 'Milk'), Support: 4
Itemset: ('Milk', 'Toothpa

Explanation of the Code:

1) Generating Itemsets: We use the itertools.combinations function to generate all possible 1-itemsets, 2-itemsets, and so on. The loop runs until no frequent itemsets are found at a particular level.

2) Counting Frequency: The function count_frequency checks how many transactions contain a given itemset. Itemsets that meet the minimum support threshold are considered frequent.

3) Generating Association Rules: We then generate association rules for the frequent itemsets using the basic "if A, then B" format. Confidence is calculated based on the proportion of transactions containing A that also contain B.

4) Stopping Criteria: The algorithm terminates when no frequent k-itemsets are found.

Output Example:

1)Frequent Itemsets: Itemsets with their support count.

2)Association Rules: Rules of the form "A -> B" along with their support and confidence values.

Brute Force Complexity:

This method is computationally expensive because it generates all possible itemsets and checks each one. However, it ensures that no frequent itemsets are missed. Optimizations like pruning can help reduce the search space in other algorithms like Apriori, but brute force ensures completeness.

1) Load the Transactional Databases: We will use the previously generated 5 transactional databases.

2) Set User-Specified Parameters: Accept user inputs for minimum support and confidence levels.

3) Brute Force Algorithm: Implement the brute-force frequent itemset generation (from the previous code) with timing measurement.

4) Apriori Algorithm: Use a pre-existing implementation from a Python library like mlxtend.

5) FP-Growth Algorithm: Use an existing package like fp-growth to generate frequent itemsets and rules.

6) Compare Results: Compare the itemsets and association rules generated by all three algorithms.

7) Performance Timing: Measure and compare the execution time of the three algorithms.

8) Display Results: Output the frequent itemsets and association rules for each algorithm, showing the differences and similarities.

In [13]:
pip install mlxtend fpgrowth_py


In [14]:
import time
import itertools
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from fpgrowth_py import fpgrowth
from collections import defaultdict

In [15]:
# Load transactions from CSV (modify as needed to load your generated datasets)
def load_transactions(filename):
    df = pd.read_csv(filename)
    transactions = df['Items'].apply(lambda x: x.split(', '))
    return transactions.tolist()

# Get user input for support and confidence
def get_user_input():
    min_support = float(input("Enter minimum support (e.g., 0.1): "))
    min_confidence = float(input("Enter minimum confidence (e.g., 0.5): "))
    return min_support, min_confidence

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# Brute-force algorithm
def brute_force(transactions, min_support):
    all_items = sorted(set(item for transaction in transactions for item in transaction))
    transaction_count = len(transactions)
    min_support_count = int(min_support * transaction_count)

    # Generate itemsets and count frequencies
    def count_frequency(itemset, transactions):
        return sum(1 for transaction in transactions if set(itemset).issubset(set(transaction)))

    frequent_itemsets = []
    k = 1
    while True:
        itemsets = list(itertools.combinations(all_items, k))
        current_frequent_itemsets = []
        for itemset in itemsets:
            frequency = count_frequency(itemset, transactions)
            if frequency >= min_support_count:
                current_frequent_itemsets.append((itemset, frequency))

        if not current_frequent_itemsets:
            break

        frequent_itemsets.extend(current_frequent_itemsets)
        k += 1

    return frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# Generate association rules for brute-force
def generate_association_rules_bruteforce(frequent_itemsets, transactions, min_confidence):
    transaction_count = len(transactions)
    rules = []
    for itemset, support in frequent_itemsets:
        if len(itemset) < 2:
            continue
        for i in range(1, len(itemset)):
            for lhs in itertools.combinations(itemset, i):
                rhs = set(itemset) - set(lhs)
                lhs_count = sum(1 for transaction in transactions if set(lhs).issubset(set(transaction)))
                confidence = support / lhs_count
                if confidence >= min_confidence:
                    rules.append((lhs, rhs, support / transaction_count, confidence))
    return rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Apriori algorithm
def run_apriori(transactions, min_support, min_confidence):
    # Convert transactions into a one-hot encoded DataFrame
    df = pd.DataFrame(transactions)
    one_hot = pd.get_dummies(df.stack()).groupby(level=0).sum()

    # Use Apriori from mlxtend
    frequent_itemsets = apriori(one_hot, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    return frequent_itemsets, rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# FP-growth algorithm
def run_fp_growth(transactions, min_support, min_confidence):
    frequent_itemsets, rules = fpgrowth(transactions, minSupRatio=min_support, minConf=min_confidence)
    return frequent_itemsets, rules

# Time execution of a function
def time_function(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
# Main function to run all algorithms and compare results
def main():
    # Load transactions from CSV (modify file names as necessary)
    transactions_list = [load_transactions(f"supermarket_transactions_1.csv") for i in range(5)]

    # Get user input for support and confidence
    min_support, min_confidence = get_user_input()

    for i, transactions in enumerate(transactions_list):
        print(f"\n--- Results for Database {i+1} ---")

        # Brute-force algorithm
        brute_force_result, brute_force_time = time_function(brute_force, transactions, min_support)
        brute_force_rules = generate_association_rules_bruteforce(brute_force_result, transactions, min_confidence)
        print(f"\nBrute-force frequent itemsets: {brute_force_result}")
        print(f"Brute-force association rules: {brute_force_rules}")
        print(f"Brute-force execution time: {brute_force_time:.4f} seconds")

        # Apriori algorithm
        apriori_result, apriori_time = time_function(run_apriori, transactions, min_support, min_confidence)
        print(f"\nApriori frequent itemsets: {apriori_result}")
        print(f"Apriori association rules: {apriori_result[1]}")
        print(f"Apriori execution time: {apriori_time:.4f} seconds")

        # FP-growth algorithm
        fp_growth_result, fp_growth_time = time_function(run_fp_growth, transactions, min_support, min_confidence)
        print(f"\nFP-Growth frequent itemsets: {fp_growth_result}")
        print(f"FP-Growth association rules: {fp_growth_result[1]}")
        print(f"FP-Growth execution time: {fp_growth_time:.4f} seconds")

        # Timing comparison
        print(f"\nTiming comparison for Database {i+1}:")
        print(f"Brute-force time: {brute_force_time:.4f} seconds")
        print(f"Apriori time: {apriori_time:.4f} seconds")
        print(f"FP-Growth time: {fp_growth_time:.4f} seconds")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enter minimum support (e.g., 0.1): 0.1
Enter minimum confidence (e.g., 0.5): 0.5

--- Results for Database 1 ---

Brute-force frequent itemsets: [(('Bananas (1kg)',), 9), (('Bread (Whole Wheat)',), 2), (('Canned Beans (400g)',), 6), (('Diapers',), 6), (('Eggs (Dozen)',), 8), (('Milk (1L)',), 7), (('Rice (5kg)',), 8), (('Shampoo',), 6), (('T-shirts',), 7), (('Toothpaste',), 4), (('Bananas (1kg)', 'Diapers'), 4), (('Bananas (1kg)', 'Eggs (Dozen)'), 5), (('Bananas (1kg)', 'Milk (1L)'), 3), (('Bananas (1kg)', 'Rice (5kg)'), 4), (('Bananas (1kg)', 'Shampoo'), 4), (('Bananas (1kg)', 'T-shirts'), 3), (('Canned Beans (400g)', 'Diapers'), 2), (('Canned Beans (400g)', 'Eggs (Dozen)'), 4), (('Canned Beans (400g)', 'Milk (1L)'), 3), (('Canned Beans (400g)', 'Rice (5kg)'), 2), (('Canned Beans (400g)', 'Shampoo'), 2), (('Diapers', 'Eggs (Dozen)'), 4), (('Diapers', 'Rice (5kg)'), 3), (('Diapers', 'Shampoo'), 2), (('Diapers', 'T-shirts'), 3), (('Eggs (Dozen)', 'Milk (1L)'), 4), (('Eggs (Dozen)', 'Rice

/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in wors

Explanation:

Load Transactions: Transactions are loaded from CSV files and passed to the three algorithms.

Brute-Force Algorithm: This algorithm generates itemsets by iterating over all combinations and counts their frequency. Association rules are generated based on user-specified support and confidence.

Apriori Algorithm: Uses the mlxtend library’s implementation. It converts transactions into a one-hot encoded DataFrame and generates itemsets and rules using Apriori.

FP-Growth Algorithm: Uses the fpgrowth_py library to generate frequent itemsets and association rules.

Timing Performance: The time_function wrapper is used to measure the execution time for each algorithm.

User Input: The user provides support and confidence thresholds, which are applied to all three algorithms.

Output:

The program will display:

Frequent Itemsets and Association Rules for each algorithm (Brute-force, Apriori, FP-Growth).

Execution Time for each algorithm.

Comparison of Results from the three algorithms.

Performance Comparison:

By running the program, you can compare:

The frequent itemsets and association rules generated by each algorithm.

The execution time of each algorithm, allowing you to determine which is faster.